In [5]:
"""
Fine-tuning the library models for causal language modeling (GPT, GPT-2, CTRL, ...) on a text file or a dataset.
Here is the full list of checkpoints on the hub that can be fine-tuned by this script:
https://huggingface.co/models?filter=text-generation
"""

"""
    Code here heavily burrows from https://github.com/locuslab/wanda/tree/main
"""
# You can also adapt this script on your own causal language modeling task. Pointers for this are left as comments.
from transformers.pytorch_utils import  find_pruneable_heads_and_indices, prune_linear_layer
import logging
import math
import os
import sys
from dataclasses import dataclass, field
from itertools import chain
from typing import Optional, List 
import pdb
import pickle as pkl
import gc
import time
from peft import PeftModel
# Uncomment this out if running the Eleuther evaluation harness
# import lm_eval
# from lm_eval import evaluator

import datasets
# import evaluate
import torch
from datasets import load_dataset, concatenate_datasets
import numpy as np
from tqdm import tqdm

import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_CAUSAL_LM_MAPPING,
    AutoConfig,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    is_torch_tpu_available,
    set_seed
)
from transformers.testing_utils import CaptureLogger
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    # prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from evaluate_ppl import evaluate_ppl

In [6]:
device_to_use = 'cuda:5'
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling


In [7]:
target_model_name = 'meta-llama/Llama-2-7b-hf'
tokenizer_name = 'meta-llama/Llama-2-7b-hf'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(target_model_name, torch_dtype=torch.float16, device_map=device_to_use)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.54s/it]


In [8]:
adapter_name = '/home/vashistt/Desktop/anlp-project/finetuned_model_prune_c4_ft_wiki/'
# model.load_adapter(adapter_name)
# model.set_active_adapters('prune-c4_ft_wiki_adapter')
model = PeftModel.from_pretrained(model, adapter_name, adapter_name="prune_c4_ft_wiki_adapter")

RuntimeError: Error(s) in loading state_dict for PeftModelForCausalLM:
	size mismatch for base_model.model.model.layers.0.self_attn.q_proj.lora_B.prune_c4_ft_wiki_adapter.weight: copying a param with shape torch.Size([1920, 64]) from checkpoint, the shape in current model is torch.Size([4096, 64]).
	size mismatch for base_model.model.model.layers.0.self_attn.k_proj.lora_B.prune_c4_ft_wiki_adapter.weight: copying a param with shape torch.Size([1920, 64]) from checkpoint, the shape in current model is torch.Size([4096, 64]).
	size mismatch for base_model.model.model.layers.0.self_attn.v_proj.lora_B.prune_c4_ft_wiki_adapter.weight: copying a param with shape torch.Size([1920, 64]) from checkpoint, the shape in current model is torch.Size([4096, 64]).
	size mismatch for base_model.model.model.layers.0.self_attn.o_proj.lora_A.prune_c4_ft_wiki_adapter.weight: copying a param with shape torch.Size([64, 1920]) from checkpoint, the shape in current model is torch.Size([64, 4096]).
	size mismatc